In [63]:
import requests as req
from bs4 import BeautifulSoup
import time
import json
import re

In [41]:
# 取新聞清單
my_headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36'
}

for i in range(1, 2):
    # 政治
    res = req.get(f'https://news.ttv.com.tw/category/國際/{i}', headers=my_headers)
    soup = BeautifulSoup(res.text, 'lxml')
    news_list = soup.select('article.container a')
    for news in news_list:
        news_title = news.select('div.title')[0].text.strip()
        news_time = news.select('div.time')[0].text.strip()
        link = news['href']
        print(f'標題: {news_title}')
        print(f'時間: {news_time}')
        print(f'連結: {link}')
        req.post('http://127.0.0.1:5000/news', json={'news_time': news_time, 'news_title': news_title, 'news_url': link, 'source_website': '台視'})
    print(f'第{i}頁結束')
    time.sleep(1)


標題: 巴基斯坦暴風雨、沙塵暴夾擊至少釀20死　南部熱浪...
時間: 2025.05.26 16:32
連結: https://news.ttv.com.tw/news/11405260024000N
標題: 台海局勢升溫 關島總督訪台會晤賴清德
時間: 2025.05.26 16:18
連結: https://news.ttv.com.tw/news/11405260001900W
標題: 刷新世界紀錄！　古巴前水球國手水中頂球31分鐘
時間: 2025.05.26 14:27
連結: https://news.ttv.com.tw/news/11405260012000I
標題: 日本擬規定嚴格檢討外國駕照更換制度　觀光自駕將不...
時間: 2025.05.26 13:04
連結: https://news.ttv.com.tw/news/11405260001400W
標題: 女中彩券頭獎24億！ 才要領錢被取消氣炸提告
時間: 2025.05.26 11:53
連結: https://news.ttv.com.tw/news/11405260000900W
標題: 川普轟普亭「瘋了」 點名澤倫斯基與拜登同樣有責
時間: 2025.05.26 10:55
連結: https://news.ttv.com.tw/news/11405260000400W
標題: 拜登自曝四期攝護腺癌 各界多所質疑
時間: 2025.05.26 10:48
連結: https://news.ttv.com.tw/news/11405260000800W
標題: 大阪飯店房費高漲 業者力拚世博商機
時間: 2025.05.26 10:35
連結: https://news.ttv.com.tw/news/11405260000700W
標題: 美中關稅戰經濟恐蕭條 廠商訂單爆滿
時間: 2025.05.26 10:21
連結: https://news.ttv.com.tw/news/11405260000600W
標題: 川普與普亭再通話 極力推進烏俄停火
時間: 2025.05.26 10:06
連結: https://news.ttv.com.tw/news/11405260000500W
標題: 俄對烏最大規模空襲！澤倫斯基籲美表態　川普回應了
時間: 2025.05.2

In [59]:
# 取得待爬清單
res = req.post('http://127.0.0.1:5000/get_wait_query_list', json={'source_website': '台視', 'count': 1})
query_list = json.loads(res.text)

In [ ]:
# 取新聞內容
my_headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36'
}

for query_data in query_list:
    res = req.get(query_data['news_url'], headers=my_headers)
    soup = BeautifulSoup(res.text, 'lxml')
    div = soup.select("div#newscontent")
    p = div[0].select("p")

    # 標題
    title = soup.select("h1.mb-ht-hf")[0].text.strip()
    title = re.sub(r'\u3000', ' ', title)

    # 抓取圖片
    soup.select("div#newscontent img")[0]

    # 內文
    content = ''
    for x in p:
        content += x.text

    # 抓取編輯
    a = re.search(r'責任編輯／([\u4e00-\u9fff]+)', content)[1]


'巴基斯坦暴風雨、沙塵暴夾擊至少釀20死\u3000南部熱浪來襲飆48度高溫'

In [86]:


ul = soup.select("ul.tag > li a")

In [ ]:
#TODO  抓tag
for x in ul:
    print(x.text)

巴基斯坦
暴風雨
沙塵暴
巴基斯坦
暴風雨
沙塵暴
死亡
災害
天災
48度
高溫
極端天氣
電力過載
斷電
國際
